# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
# Import libraries for context-sensitive spelling corrector using N-gram language models
import re
import collections
import nltk
import numpy as np
from nltk.util import ngrams
from collections import Counter, defaultdict

In [2]:
from collections import defaultdict

def load_ngrams(file_path):
    """Read N-grams from dataset file"""
    n_grams = defaultdict(float) # Use defaultdict to handle missing keys
    total_n_grams = 0
    with open(file_path, 'r') as f:
        for line in f:
            count, ngram = float(re.findall(r"\w+", line)[0]), '|'.join([l for l in re.findall(r"\w+", line)[1:]]) # Split line into count and n-gram
            n_grams[ngram] += count # Update the count for the n-gram
            total_n_grams += count
        
    for ngram in n_grams:
        n_grams[ngram] /= total_n_grams
    return n_grams

# Load bigram and 5-gram data
bigram_data = load_ngrams("data/w2_.txt")
fivegram_data = load_ngrams("data/w5_.txt")

# Merge bigram and 5-gram data
combined_ngrams = defaultdict(float)
combined_ngrams.update(bigram_data)
combined_ngrams.update(fivegram_data)


In [57]:
print(f"Total N-grams: {len(combined_ngrams)}")
# Count the words in combined_ngrams
words = Counter('|'.join(combined_ngrams).split('|'))
print(f"Total words: {len(words)}")

# Normalize frequencies by total word count
total_count = sum(words.values())
for word in words:
    combined_ngrams[word] = words[word] / total_count


Total N-grams: 2128532
Total words: 64296


In [4]:
import math

# Define the keyboard layout
keyboard = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def calculate_distance(key1, key2):
    """Calculate the Euclidean distance between two keys on the keyboard."""
    if key1 == '|':
        x1, y1 = -1, -1
    else:
        x1, y1 = keyboard[key1]
    if key2 == '|':
        x2, y2 = -1, -1
    else:
        x2, y2 = keyboard[key2]
    return np.abs(x1 - x2) + np.abs(y1 - y2)

# Test the function
assert calculate_distance("z", "z") == 0
assert calculate_distance("q", "p") == len("qwertyuiop") - 1
assert calculate_distance("q", "m") == 8
print(calculate_distance('o', 'a'))

9


In [5]:
# Edit distance algorithm Based on the Damerau-Levenshtein distance (https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/)
def edit_distance(s1, s2, is_cost=False):
    """Calculate the Damerau-Levenshtein distance between two strings."""
    if len(s1) > len(s2):
        return edit_distance(s2, s1, is_cost)

    if len(s2) == 0:
        return len(s1)

    def del_cost(a, b):
        if not is_cost:
            return 1
        else:
            return calculate_distance(a, b) if a!=b else 0.5

    def ins_cost(a, b):
        if not is_cost:
            return 1
        else:
            return calculate_distance(a, b) if a!=b else 0.5

    def sub_cost(a, b):
        return calculate_distance(a, b) if is_cost else 1

    d = np.zeros((len(s1)+1, len(s2)+1))
    for i in range(len(s1)+1):
        d[i][0] = i
    for j in range(len(s2)+1):
        d[0][j] = j
    

    for i in range(1, len(s1)+1):
        for j in range(1, len(s2)+1):
            c1 = s1[i-1]
            c2 = s2[j-1]

            insertions = d[i][j-1] + ins_cost(c1, c2)
            deletions = d[i-1][j] + del_cost(c1, c2)
            substitutions = d[i-1][j-1] + sub_cost(c1, c2)*(1 if c1!=c2 else 0)
            d[i][j] = min(insertions, deletions, substitutions)

            x1 = s1[i-2]
            x2 = s2[j-2]

            if i>2 and j>2 and c1 == x2 and c2 == x1:
                d[i, j] = np.min([d[i, j], d[i - 3][j - 3] + 1])

    return d[-1][-1]


# Test the function
assert edit_distance("alright", "alrigkt") == 1
assert edit_distance('', 'Kazan') == 5
assert edit_distance('Foreign', 'foregin') == 2
assert edit_distance('kingsdom', 'knigsdoom', True) == 1.5
assert edit_distance('love', 'lave', True) == 5.5


In [7]:
def find_word(w, n, words):
    for word in words:
        state = (range(n+1), range(n+1))
        for c in word: 
            idx, values = state
            if idx and idx[0] == 0 and values[0] < n:
                new_idx = [0]
                new_values = [values[0] + 1]
            else:
                new_idx = []
                new_values = []

            for j,i in enumerate(idx):
                if i == len(w): break
                cost = 0 if w[i] == c else 1
                val = values[j] + cost
                if new_idx and new_idx[-1] == i:
                    val = min(val, new_values[-1] + 1)
                if j+1 < len(idx) and idx[j+1] == i+1:
                    val = min(val, values[j+1] + 1)
                if val <= n:
                    new_idx.append(i+1)
                    new_values.append(val)
                else:
                    continue

            state = (new_idx, new_values)
        if bool(state[0]) and state[0][-1] == len(w): 
            yield word

In [19]:
def list_edits_using_n_gram(n_gram, n=2, edits=2):
    n_grams_ = ['|'.join(n_gram[i:i+n]) for i in range(len(n_gram) - n + 1)]
    g_edits = [defaultdict(lambda: 0) for _ in n_gram]

    for i, x in enumerate(n_grams_):
        edits_ = list(map(lambda y: y.split('|'), find_word(x, edits, combined_ngrams)))
        for edit in edits_:
            for j, token in enumerate(edit):
                g_edits[i + j][token] += combined_ngrams['|'.join(edit)]
    res = [dict(x) for x in g_edits]
    return res

In [30]:
list_edits_using_n_gram(["i", "am", "giong", "to", "the", "park"], n=5, edits=3)

[{'i': 1.7019393599006068e-06},
 {'am': 1.7019393599006068e-06},
 {'going': 1.7019393599006068e-06},
 {'to': 1.7019393599006068e-06},
 {'the': 1.7019393599006068e-06},
 {}]

In [15]:
list_edits_using_n_gram(["i", "cookd", "fortuna", "cookis"], n=2, edits=2)

[{'a': 2.409695644420373e-06,
  'i': 0.0004278343127868501,
  'in': 1.0496648176129264e-06,
  'is': 1.1996169344147731e-06,
  'it': 6.800154134037232e-07,
  'j': 1.2902856562019363e-07},
 {'cook': 4.191684753391155e-06,
  'cooked': 2.2283582008460467e-06,
  'booked': 2.092355118165302e-07,
  'choke': 1.2205404855964261e-07,
  'choked': 3.347768189064483e-07,
  'cocked': 8.718146325688759e-08,
  'could': 0.0003057942132613286,
  'good': 1.1507953149909161e-07,
  'hook': 9.76432388477141e-08,
  'hooked': 4.98677969829397e-07,
  'look': 3.421349344053296e-05,
  'looked': 4.231788226489323e-05,
  'took': 4.2663120859390504e-05,
  'cooks': 4.289327992238869e-07},
 {'fortune': 5.47499589253254e-07},
 {'cookie': 3.243150433156218e-07, 'cookies': 2.2318454593763221e-07}]

In [35]:
def correct_sentence(sentence, n=2, edits=2):
    # Разбиваем предложение на слова
    words = re.findall(r"\w+", sentence.lower())
    
    # Получаем возможные исправления и их оценки
    word = "|".join(words)
    corrections = np.array(list_edits_using_n_gram(words, n, edits), dtype=object)
    
    # Применяем исправления к словам
    corrected_words = []
    for i, word in enumerate(words):
        # Выбираем наиболее подходящее исправление для текущего слова
        best_correction = None
        best_score = -1
        for correction, score in corrections[i].items():
            if score > best_score:
                best_correction = correction
                best_score = score
        
        # Если для слова найдено исправление, применяем его
        if best_correction:
            corrected_words.append(best_correction)
        else:
            corrected_words.append(word)
    
    # Возвращаем исправленное предложение
    return ' '.join(corrected_words)



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

The way I try to fix mistakes in grammar is by looking at groups of words. I look at groups of 5 words, 2 words, and just one word. I found that using just one size of group was not enough. If I use too small of a group, the code might not understand the whole meaning. But if I use too big of a group, the code might not find enough examples to help fix the mistake. I made a system that helps choose the best word to replace another word. It looks at how often different words can be used and also how similar they are to the original word.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [78]:
# A function that compares two texts and returns 
# the number of matches and differences
from itertools import zip_longest

def compare(text1, text2):
    l1 = text1.lower().split()
    l2 = text2.lower().split()
    good = 0
    bad = 0
    for word1, word2 in zip_longest(l1, l2, fillvalue=''):
        if word1 != word2:
            bad += 1
        else:
            good += 1
    return (good, bad)

# Helper function to calculate the percentage of misspelled words
def percentageOfBad(x):
    return (x[1] / (x[0] + x[1])) * 100

def test_func(corrects, incorrects):
    err = 0
    res = []
    for corct, incorrect in zip(corrects, incorrects):
        if len(incorrect) > 5:
            corrected = correct_sentence(incorrect, n=5, edits=3)
            # corrected = correct_sentence(corrected, n=2, edits=2)
        else:
            corrected = correct_sentence(incorrect, n=2, edits=2)
        # corrected = correct_sentence(corrected, n=1, edits=2)

        err = compare(corrected, corct)
        perc = percentageOfBad(err)
        res.append((perc, corrected))
    return res

# print(test_func(["I am going to the park"], ["I am giong to the park"]))

In [79]:
# lists of correct sentences and its incorrect versions
corrects = ["I am going to the park", "I don't like to play football", "I love to cook and bake", "He is very tall", "She is very beautiful",
            "We are going to the cinema tonight", "The sun is shining brightly", "He is my best friend", "I don't know what to do", "I have a cat"
            "We are eating pizza", "She is reading a book", "The quick brown fox jumps over the lazy dog", "I am a student", "I love to eat apples"]
incorrects = ["I am giong to th park", "I don't lke to paly football", "I lpve to cok and bske", "Hhe is veri tal", "Sh ise vary butiful",
            "We are ging to the cnema tonit", "The sun is shinng brighty", "He is my bst freind", "I dont kbow wgat to do", "I hve a ct"
            "We are eat pizza", "She is reading a bok", "The qwick brow fox jump over the lazy dog", "I m a stdent", "I lovee too eat aples"]

ans_my = test_func(corrects, incorrects)
print(ans_my)

In [ ]:
print(sum(ans_my[i][0] for i in range(len(ans_my))) / len(ans_my))

44.42743764172336


### Norvig Solution

In [70]:
import re
from collections import Counter
import re
from collections import Counter

def P(word, N=sum(words.values())): 
    "Probability of `word`."
    return words[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words_): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words_ if w in words)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдеёжзийклмнопрстуфхцчшщьыъэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvig(corrects, incorrects):
    corrects = [re.findall("\w+", i.lower()) for i in corrects]
    incorrects = [re.findall("\w+", i.lower()) for i in incorrects]
    err = 0
    res = []
    for corct, incorrect in zip(corrects, incorrects):
        corrected = []
        for i in range(len(incorrect)):
            corrected.append(correction(incorrect[i]) )
        
        err = compare(" ".join(corrected), " ".join(corct))
        perc = percentageOfBad(err)
        res.append((perc, corrected))
    return res

ans_norvig = norvig(corrects, incorrects)
print(ans_norvig)
print(sum(ans_norvig[i][0] for i in range(len(ans_norvig))) / len(ans_norvig))

[(16.666666666666664, ['i', 'am', 'going', 'to', 'th', 'park']), (28.57142857142857, ['i', 'don', 't', 'le', 'to', 'pay', 'football']), (50.0, ['i', 'le', 'to', 'co', 'and', 'bse']), (50.0, ['he', 'is', 'ver', 'tal']), (100.0, ['sh', 'ise', 'vary', 'butiful']), (42.857142857142854, ['we', 'are', 'ing', 'to', 'the', 'nea', 'toni']), (40.0, ['the', 'sun', 'is', 'shinn', 'bright']), (20.0, ['he', 'is', 'my', 'st', 'friend']), (85.71428571428571, ['i', 'dont', 'bow', 'wat', 'to', 'do']), (42.857142857142854, ['i', 'he', 'a', 'cte', 'are', 'eat', 'pizza']), (20.0, ['she', 'is', 'reading', 'a', 'bok']), (33.33333333333333, ['the', 'wick', 'brow', 'fox', 'jump', 'over', 'the', 'lazy', 'dog']), (50.0, ['i', 'm', 'a', 'stent']), (40.0, ['i', 'love', 'too', 'eat', 'apes'])]
44.285714285714285
